In [1]:
import socket 
from _thread import *
import sys
from collections import defaultdict as df
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
import time
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# print(msg)
model = pickle.load(open("CNNBILSTM.pkl", 'rb'))

In [3]:
class Server:
    def __init__(self):
        self.rooms = df(list)
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)


    def accept_connections(self, ip_address, port):
        self.ip_address = ip_address
        self.port = port
        self.server.bind((self.ip_address, int(self.port)))
        self.server.listen(100)
        print("Server is ready and listening for connections...")
        while True:
            connection, address = self.server.accept()
            print(str(address[0]) + ":" + str(address[1]) + " Connected")

            start_new_thread(self.clientThread, (connection,))

        self.server.close()

    
    def clientThread(self, connection):
        user_id = connection.recv(1024).decode().replace("User  ", "")  # Indented inside the function
        room_id = connection.recv(1024).decode().replace("Join ", "")
    
        print(f"User  {user_id} joined room {room_id}")  # Log user joining
    
        if room_id not in self.rooms:
            connection.send("New Group created".encode())
        else:
            connection.send("Welcome to chat room".encode())
    
        connection.send("Server is ready".encode())  # Send server ready message to client
    
        self.rooms[room_id].append(connection)
    
        while True:
            try:
                message = connection.recv(1024)
                pred = 0  # Default value, assuming non-bullying
                if message:
                    print(f"Message received from {user_id}: {message.decode()}")  # Log received message
    
                    # Process the message
                    self.broadcast(message.decode(), connection, room_id, pred)
                else:
                    self.remove(connection, room_id)
                    break
            except:
                self.remove(connection, room_id)
                break

    def broadcast(self, message_to_send, connection, room_id, pred):
        print(f"Broadcasting message in room {room_id}: {message_to_send}")  # Log broadcasting
        for client in self.rooms[room_id]:
            if client != connection:
                try:
                    if pred == 0:  # Non-bullying message
                        client.send(message_to_send.encode())
                        print("Message sent to client")  # Log successful send
                    else:
                        client.send("Bullying message detected and hidden".encode())
                except:
                    client.close()
                    self.remove(client, room_id)

        # print("Sent")

    def prettyPrinter(self,data_1):
         # List of stopwords 
        my_file = open("stopwords.txt", "r")
        content = my_file.read()
        content_list = content.split("\n")
        my_file.close()
        tfidf_vector =  TfidfVectorizer(stop_words = content_list, lowercase = True,vocabulary=pickle.load(open("tfidf_vector_vocabulary.pkl", "rb")))
        data_2=tfidf_vector.fit_transform([data_1])
        print(data_2)
        pred = model.predict(data_2)
        print(pred)
        if pred==0:
            print('Non bullying')
            return pred
        else: 
            print("Stop bullying people and behave decently.")
            return pred
   
    def remove(self, connection, room_id):
        if connection in self.rooms[room_id]:
            self.rooms[room_id].remove(connection)

In [ ]:
if __name__ == "__main__":
    ip_address = "0.0.0.0"
    port = 12345

    s = Server()
    s.accept_connections(ip_address, port)

Server is ready and listening for connections...
10.255.34.63:60815 Connected
User  Qwerty joined room 123
Message received from Qwerty: Hy there
Broadcasting message in room 123: Hy there
